In [ ]:
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install scikit-learn
%pip install openpyxl
%pip install xgboost
%pip install sklearn.preprocessing
#pip install catboost

In [29]:
# Import the required packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#Google Sheet Link: https://docs.google.com/spreadsheets/d/1RiU9c3YihCsAqh18C25WMfTxkOiAf-nXhxAWtEaKLA0/edit#gid=0
pd.set_option('display.max_rows', 200, 'display.min_rows', 30) #Displays a max of 200 rows for testing

sheet_id = '1RiU9c3YihCsAqh18C25WMfTxkOiAf-nXhxAWtEaKLA0'
xls = pd.ExcelFile(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=xlsx")

arrivals = pd.read_excel(xls, 'Arrivals', header = 0) #Reads in the Google Sheets Data
departures = pd.read_excel(xls, 'Departures', header = 0)
arrivals = arrivals.drop(['blank', 'Communication Type/Reason for Speaking.1'], axis=1) #Drops unnecessary Columns/Headers
departures = departures.drop(['Airport', 'Communication Type/Reason for Speaking.1'], axis=1)

arrivals = arrivals.dropna(thresh=14) #Drops all rows and columns with more than 14 N/A values
departures = departures.dropna(thresh=14)

# create a new leftmost column for the index
# iterate through the rows and index the responses of each flight based on unique flight no.
counter = 0
flight_no = -1
arrivals.insert(loc = 0, # create arrivals indexing
                column = 'index',
                value = 0)
for row in arrivals.iterrows():
    if flight_no != row[1]['Flight No.']:
        flight_no = row[1]['Flight No.']
        counter = 0
    arrivals.loc[row[0], 'index'] = counter
    counter += 1

counter = 0
flight_no = -1
departures.insert(loc = 0, # create departures indexing
                column = 'index',
                value = 0)
for row in departures.iterrows():
    if flight_no != row[1]['Flight No.']:
        flight_no = row[1]['Flight No.']
        counter = 0
    departures.loc[row[0], 'index'] = counter
    counter += 1


#Gets the specifics columns from collected data
x = arrivals[['index']]
y = arrivals[['Communication Type/Reason for Speaking']]

# Change y-axis labels to numeric values
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y = encoder.fit_transform(y.values.ravel())

#split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = .25, random_state = 42)

import xgboost as xgb
from sklearn.model_selection  import GridSearchCV

model = xgb.XGBClassifier()

# set up for grid search for hyperparameter tuning
optimization_dict = {'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                     'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 50],
                     'learning_rate': [0.2, 0.25, 0.3, 0.35, 0.5, 0.75, 0.99]
                     }

xgb_model = GridSearchCV(model, optimization_dict, scoring='accuracy', verbose=1)
xgb_model.fit(x_train, y_train)

from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, x, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

from sklearn.metrics import accuracy_score

y_pred = xgb_model.predict(x_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (acc * 100.0))
print(xgb_model.best_params_)
print(xgb_model.best_score_)

Fitting 5 folds for each of 1120 candidates, totalling 5600 fits


C:\Users\yangh\Documents\ASU\Year3\Spring23\CSE486\honeywell-atc-ai\venv\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Accuracy: 0.48 (+/- 0.23)
Accuracy: 53.41%
{'learning_rate': 0.2, 'max_depth': 8, 'n_estimators': 4}
0.5173439767779391


NotFittedError: need to call fit or load_model beforehand

In [26]:
from catboost import CatBoostClassifier

cat_model = CatBoostClassifier()
optimization_dict = {'max_depth': [2],
                     'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 50],
                     'learning_rate': [0.2, 0.25, 0.3, 0.35]
                     }

# set up for grid search for hyperparameter tuning
cat_model = GridSearchCV(cat_model, optimization_dict, scoring='accuracy', verbose=1)
cat_model.fit(x_train, y_train)

y_pred = cat_model.predict(x_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (acc * 100.0))
print(cat_model.best_params_)
print(cat_model.best_score_)

cat_predictions = cat_model.predict(new_x)

Fitting 5 folds for each of 64 candidates, totalling 320 fits
0:	learn: 1.8459108	total: 364us	remaining: 0us
0:	learn: 1.8499927	total: 384us	remaining: 0us
0:	learn: 1.8487328	total: 369us	remaining: 0us
0:	learn: 1.8521694	total: 366us	remaining: 0us
0:	learn: 1.8293512	total: 389us	remaining: 0us
0:	learn: 1.8459108	total: 330us	remaining: 330us
1:	learn: 1.7356952	total: 613us	remaining: 0us
0:	learn: 1.8499927	total: 353us	remaining: 353us
1:	learn: 1.7376154	total: 687us	remaining: 0us
0:	learn: 1.8487328	total: 362us	remaining: 362us
1:	learn: 1.7375890	total: 671us	remaining: 0us
0:	learn: 1.8521694	total: 373us	remaining: 373us
1:	learn: 1.7400575	total: 744us	remaining: 0us


C:\Users\yangh\Documents\ASU\Year3\Spring23\CSE486\honeywell-atc-ai\venv\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


0:	learn: 1.8293512	total: 378us	remaining: 378us
1:	learn: 1.7602893	total: 693us	remaining: 0us
0:	learn: 1.8459108	total: 404us	remaining: 809us
1:	learn: 1.7356952	total: 692us	remaining: 346us
2:	learn: 1.6537316	total: 1.02ms	remaining: 0us
0:	learn: 1.8499927	total: 330us	remaining: 661us
1:	learn: 1.7376154	total: 660us	remaining: 330us
2:	learn: 1.6426729	total: 942us	remaining: 0us
0:	learn: 1.8487328	total: 334us	remaining: 669us
1:	learn: 1.7375890	total: 604us	remaining: 302us
2:	learn: 1.6439992	total: 907us	remaining: 0us
0:	learn: 1.8521694	total: 376us	remaining: 753us
1:	learn: 1.7400575	total: 707us	remaining: 353us
2:	learn: 1.6512372	total: 1.06ms	remaining: 0us
0:	learn: 1.8293512	total: 312us	remaining: 624us
1:	learn: 1.7602893	total: 683us	remaining: 341us
2:	learn: 1.6652370	total: 1.07ms	remaining: 0us
0:	learn: 1.8459108	total: 320us	remaining: 962us
1:	learn: 1.7356952	total: 635us	remaining: 635us
2:	learn: 1.6537316	total: 909us	remaining: 303us
3:	learn: